## 1. Install Dependencies

In [ ]:
# Dependencies already installed via terminal
# If needed, run in terminal: 
# /Users/saritajoshi/testbedmanagement/.venv/bin/python -m pip install neo4j google-adk python-dotenv --index-url https://pypi.org/simple/
print("Dependencies ready! ✓")

ERROR: Could not find a version that satisfies the requirement neo4j (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for neo4j


## 2. Configuration

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# Neo4j Configuration - Default to local Docker instance
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "password123")

# Google API Key for ADK
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")

print(f"Neo4j URI: {NEO4J_URI}")
print(f"Google API Key configured: {bool(GOOGLE_API_KEY)}")

Neo4j URI: bolt://localhost:7687
Google API Key configured: True


## 3. Local Neo4j Setup (Docker)

Run this command in terminal to start Neo4j locally:
```bash
docker run -d --name neo4j-healthcare \
  -p 7474:7474 -p 7687:7687 \
  -e NEO4J_AUTH=neo4j/password123 \
  neo4j:latest
```

In [3]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def run_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters or {})
            return [record.data() for record in result]

# Test connection
try:
    conn = Neo4jConnection(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    result = conn.run_query("RETURN 'Connected!' AS status")
    print(result[0]['status'])
except Exception as e:
    print(f"Connection failed: {e}")
    print("Please start Neo4j Docker container first.")

Connected!


## 4. Healthcare Schema Creation

In [4]:
# Clear existing data and create constraints
schema_queries = [
    "MATCH (n) DETACH DELETE n",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (h:Hospital) REQUIRE h.id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (d:Department) REQUIRE d.id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (b:Bed) REQUIRE b.id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Patient) REQUIRE p.id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (ph:Physician) REQUIRE ph.id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (i:InventoryItem) REQUIRE i.id IS UNIQUE",
]

for query in schema_queries:
    try:
        conn.run_query(query)
        print(f"Executed: {query[:50]}...")
    except Exception as e:
        print(f"Note: {e}")

print("\nSchema setup complete!")

Executed: MATCH (n) DETACH DELETE n...
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (h:Hospital) R...
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (d:Department)...
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (b:Bed) REQUIR...
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (p:Patient) RE...
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (ph:Physician)...
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (i:InventoryIt...

Schema setup complete!
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (b:Bed) REQUIR...
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (p:Patient) RE...
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (ph:Physician)...
Executed: CREATE CONSTRAINT IF NOT EXISTS FOR (i:InventoryIt...

Schema setup complete!


## 5. Schema Visualization

```
Hospital --[HAS_DEPARTMENT]--> Department
Department --[HAS_BED]--> Bed
Patient --[ADMITTED_TO]--> Bed
Patient --[HAS_CONDITION]--> Condition
Physician --[ASSIGNED_TO]--> Department
Physician --[TREATS]--> Patient
Department --[STOCKS]--> InventoryItem
```

In [5]:
# Save connection config for other notebooks
import json

config = {
    "neo4j_uri": NEO4J_URI,
    "neo4j_user": NEO4J_USER,
    "neo4j_password": NEO4J_PASSWORD,
    "google_api_key": GOOGLE_API_KEY
}

with open("healthcare_config.json", "w") as f:
    json.dump(config, f)

print("Configuration saved. Proceed to Part 2 for data population.")

Configuration saved. Proceed to Part 2 for data population.
